In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\DELL\\Desktop\\MLflow-deployment'

In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [4]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    local_data_file: Path
    unzip_data_dir: Path
    all_schema: dict
    status_file: Path

    

In [6]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories

In [7]:
# configuration
class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
        schema_file_path: Path = SCHEMA_FILE_PATH,
    ) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories([Path(self.config["artifacts_root"])])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config["data_validation"]
        schema = dict(self.schema["COLUMNS"])
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=Path(config.root_dir),
            local_data_file=Path(self.config["data_ingestion"]["local_data_file"]),  # from ingestion section
            unzip_data_dir=Path(config.unzip_data_dir),
            all_schema=schema,
            status_file=Path(config.status_file),
        )
        return data_validation_config



In [8]:
import os
from src.mlproject import logger



In [9]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:   # renamed
        try:
            validation_status = None
            
            data = pd.read_csv(self.config.local_data_file)
            all_cols = list(data.columns)
            schema_cols = self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in schema_cols:
                    validation_status = False
                    with open(self.config.status_file, "w") as f:
                        f.write(f"Data Validation Failed: {col} not in schema")
                    break
            else:
                validation_status = True
                with open(self.config.status_file, "w") as f:
                    f.write("Data Validation Successful: all columns are validated")

            return validation_status
        
        except Exception as e:
            raise e


In [13]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

2025-10-04 21:04:04,951 - INFO - yaml file: config\config.yaml loaded successfully
2025-10-04 21:04:04,965 - INFO - yaml file: params.yaml loaded successfully
2025-10-04 21:04:04,977 - INFO - yaml file: schema.yaml loaded successfully
2025-10-04 21:04:04,984 - INFO - Directory created at: artifacts
2025-10-04 21:04:04,989 - INFO - Directory created at: artifacts/data_validation
